# Setup / Imports

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

# Split the ebook into proper chapters

In [3]:
from ebooklib import epub

file1 = "../data/frankenstein.epub"
book1 = epub.read_epub(file1, options={'ignore_ncx': False})

In [4]:
from ebook_tree_parser.toctree import TocTree

In [5]:
tree = TocTree(book1)
tree

TOC Tree: title=root, href=#, level=0, #children=32

# Import openai and get the API key

In [53]:


# api key from .env file
import os
from dotenv import load_dotenv
load_dotenv(override=True)

# from openai import OpenAI
# client = OpenAI(
#   organization='org-ZmPsjhKQZsIYHP2tf06Qw0tb',
#   project='proj_7IW3lRCW3PRtkrALT3OQ7okH',
#   api_key=os.getenv("OPENAI_API_KEY")
# )
# test a call to the API 
# completion = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "user", "content": "write a haiku about ai"}
#     ]
# )
# print(completion.choices[0].message.content)

import anthropic
client = anthropic.Anthropic(
  api_key=os.getenv("ANTHROPIC_API_KEY"),
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
print(message.content)

[TextBlock(text="Briny waves crash and recede,\nEarth's tears, eons old indeed.\nMinerals dissolved, a saline brew,\nNature's alchemy, forever true.", type='text')]


In [128]:
import json
def parse_json(response:str, json_index:int=0) -> dict:
    # json is between ``` and ``` in the response, possibly with a ```json and ``` as well
    # the json_index json surrounded by ``` is the one we want
    # repeatedly find ``` until we find the json_index-th one
    # use response.find
    """ Find the json_index-th json in the response

    Args:
        response (str): The string response from the API
        json_index (int, optional): index of the json to find. Defaults to 0.

    Returns:
        dict: The json object. Could error if the json is not valid
    
    Throws:
        ValueError: If the json is not
    """
    start = -1
    count = 0
    while count < json_index+1:
        start = response.find("```", start+1)
        if start == -1:
            break
        count += 1
    # find the end of the json
    end = response.find("```", start+1)
 
    if start == -1 or end == -1:
        return None
    
    # handle the case where there is a ```json\n and ``` in the response
    if response[start+3:start+7] == "json":
        start += 7

    return json.loads(response[start:end])
    
parse_json('rw```json\n{"gi":"ffi"}```')

{'gi': 'ffi'}

# Prepare folders to store outputs

In [127]:
# make directory for the output, if it exists, create a new one with incremented number
import os

output_dir = "../output/frankenstein"
if os.path.exists(output_dir):
    i = 1
    while os.path.exists(output_dir + "_" + str(i)):
        i += 1
    output_dir += "_" + str(i)
os.makedirs(output_dir)

# make folders "inprompt", "json", and "response" folders in the output directory
os.makedirs(os.path.join(output_dir, "inprompt"))
os.makedirs(os.path.join(output_dir, "json"))
os.makedirs(os.path.join(output_dir, "response"))

In [57]:
# message = client.messages.create(
#     model="claude-3-5-sonnet-20240620",
#     max_tokens=1000,
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "text",
#                     "text": template(tree[0].content)
#                 }
#             ]
#         }
#     ]
# )
# print(message.content[0].text)

Here's the JSON created based on the provided excerpt and schema:

```json
{
  "novelInfo": {
    "title": "Frankenstein; Or, The Modern Prometheus",
    "author": "Mary Wollstonecraft Shelley",
    "genre": ["Gothic", "Science Fiction", "Horror"]
  },
  "summary": "Frankenstein; Or, The Modern Prometheus is a novel that explores themes of scientific ambition, the consequences of creating life, and the responsibilities of a creator to their creation. The story likely follows the journey of a scientist who creates a living being, leading to unforeseen and potentially disastrous consequences.",
  "characters": [
    {
      "name": "Victor Frankenstein",
      "description": "Presumed to be the main character, a scientist who likely creates artificial life.",
      "role": "protagonist"
    },
    {
      "name": "The Creature",
      "description": "Presumed to be the artificial being created by Victor Frankenstein.",
      "role": "antagonist"
    }
  ],
  "plotPoints": [],
  "worldBui

In [46]:
# completion = client.chat.completions.create(
#     model="chatgpt-4o-latest",
#     messages=[
#         {"role": "user", "content": template(tree[0].content)}
#     ]
# )
# print(completion.choices[0].message.content)

In [129]:
from novelinsights.prompts.fiction import template

prev_json = None
for i, excerpt in enumerate(tree):
    print(f"[{i}]\t{excerpt.title}")
    
    cur_template = template(excerpt, prev_json=prev_json)
    
    with open(os.path.join(output_dir, "inprompt", f"[{i}] {excerpt.title}.txt"), "w") as f:
        f.write(cur_template)
    
        message = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=8192,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": cur_template
                        }
                    ]
                }
            ]
        )
        
        with open(os.path.join(output_dir, "response", f"[{i}] {excerpt.title}.txt"), "w") as f:
            f.write(message.content[0].text)
        
        try:
            prev_json = parse_json(message.content[0].text)
        except:
            pass
        
        with open(os.path.join(output_dir, "json", f"[{i}] {excerpt.title}.json"), "w") as f:
            f.write(json.dumps(prev_json, indent=4))
            

[0]	Frankenstein;
[1]	or, the Modern Prometheus
[2]	CONTENTS
[3]	Letter 1
[4]	Letter 2
[5]	Letter 3
[6]	Letter 4
[7]	Chapter 1
[8]	Chapter 2
[9]	Chapter 3
[10]	Chapter 4
[11]	Chapter 5
[12]	Chapter 6


KeyboardInterrupt: 